### Install and import modules

In [1]:
import numpy as np
import pandas as pd
from ipynb.fs.full.plot import generate_plot
from ipynb.fs.full.preprocessing import *
from ipynb.fs.full.random_forest import *

### Train model and forecast time to earthquakes

In [2]:
training_set = pd.read_csv('C:/data/train.csv', dtype={'acoustic_data': np.int16, 'time_to_failure': np.float64})

In [3]:
#generate_plot(training_set[:9_000_000])
generate_plot(training_set)
print('Plot Generated')

C:\Users\MyPC\Anaconda3\lib\site-packages\matplotlib\cbook\__init__.py:424: MatplotlibDeprecationWarning: 
Passing one of 'on', 'true', 'off', 'false' as a boolean is deprecated; use an actual boolean (True/False) instead.
  warn_deprecated("2.2", "Passing one of 'on', 'true', 'off', 'false' as a "


Plot Generated


In [4]:
# Add statistical summaries and do segmentation
failure_times = get_failure_times(training_set, 150_000)

In [5]:
failure_times.head()

,mean,std,min,max,abs_mean,abs_std,abs_min,abs_max,q95,q99,...,std_int_mean1000,min_int_mean1000,max_int_mean1000,q95_int_mean1000,q99_int_mean1000,q05_int_mean1000,q01_int_mean1000,change_abs_int_mean1000,change_rate_int_mean1000,time_to_failure
0,4.884113,5.101106,-98.0,104.0,5.576567,4.333325,0.0,104.0,11.0,18.0,...,0.295715,3.896,5.629,5.338,5.484,4.379,4.072,74222.343443,5.629,1.430797
1,4.725767,6.588824,-154.0,181.0,5.734167,5.732777,0.0,181.0,12.0,21.0,...,0.231587,3.412,5.667,5.066,5.223,4.345,4.233,74364.426478,5.667,1.391499
2,4.906393,6.967397,-106.0,140.0,6.152647,5.895945,0.0,140.0,13.0,26.0,...,0.267012,4.055,5.957,5.344,5.486,4.446,4.239,74594.984584,5.957,1.353196
3,4.902240,6.922305,-199.0,197.0,5.933960,6.061214,0.0,199.0,12.0,22.0,...,0.266700,3.722,5.858,5.317,5.453,4.433,4.304,74333.143623,5.858,1.313798
4,4.908720,7.301110,-126.0,145.0,6.110587,6.329485,0.0,145.0,12.0,26.0,...,0.228005,3.918,6.078,5.306,5.491,4.543,4.434,74428.435969,6.078,1.274400


In [6]:
training_set = failure_times.values
feature_count = training_set.shape[-1]-1

In [7]:
rf = RF(feature_count)

In [8]:
# Min Max Scaling
feature_train, target_train = rf.generate_feature_and_target(training_set)

In [9]:
%%time
# Grid Search: best_rf
# Best RF: best_estimator
best_rf, best_estimator = rf.best_rf(training_set)

Wall time: 10h 26min 25s


In [10]:
best_estimator.fit(feature_train, target_train)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=10,
           max_features=15, max_leaf_nodes=None, min_impurity_decrease=0.0,
           min_impurity_split=None, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=1000, n_jobs=None, oob_score=False,
           random_state=None, verbose=0, warm_start=False)

In [11]:
# Add statistical summaried to test segments
feature_test = generate_test_feature()

In [12]:
forecast(best_estimator, rf.scaler, feature_test)